<a href="https://colab.research.google.com/github/SanjayS2348553/LLM/blob/main/2348553_SANJAY_S_LLM_LAB_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch